# Notes
- The original authors build univariate models
    - Created one multivariate model per variable in dataset
    - Computed mse and mae based on all of their results
- Reference articles ceated univariate and multivariate models.
    - The univariate models were only build with the OT variable
- The splits assume a month is 30 days long

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from typing import Tuple, Iterator, List
from tqdm import tqdm

In [26]:
class DataLoader:
    def __init__(self, filepath:str):
        self.filepath = filepath
        self.dataset_name = self.filepath.split("/")[-1].split(".")[0]
        self.df = self.load_data(self.filepath)
        self.columns = self.df.columns
    
    def load_data(self, filepath:str) -> pd.DataFrame:
        df = pd.read_csv(filepath)
        df = df.drop(columns="date")
        return df

    def get_train_test_data(self, df:pd.DataFrame, input_size = 672, padding=True) -> Tuple[pd.DataFrame, pd.DataFrame]:
        n, d = df.shape
        breakpoint = round(n*0.8)
        train = df.iloc[0:breakpoint]
        test = df.iloc[breakpoint-input_size:] if padding else df.iloc[breakpoint:]
        return train, test
    
    def get_lag_response_and_dependent(self, df: pd.DataFrame, input_size: int, horizon_size:int)-> pd.DataFrame:
        n, d = df.shape
        w = input_size
        h = horizon_size
        X = np.zeros([n + 1 - w - h, d*w])
        y = np.zeros([n + 1 - w - h, d*h])
        x_flat = df.values.flatten()
        for i in range(0,n + 1 - w - h):
            X[i] = x_flat[i*d:d*(i+w)]
            y[i] = x_flat[d*(i+w):d*(i+w) + d*h]
        return X, y

    def build_dataset(self, input_size = 672, horizon_size = 24, cut_off=0.75) -> Iterator[Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]]:
        df = self.df.copy()
        #Build train and test dataset
        train, test = self.get_train_test_data(df, input_size = input_size)

        sc = StandardScaler()
        cut_off = round(len(train)*cut_off)
        self.columns = train.columns
        sc.fit(train.iloc[0:cut_off])
        train = pd.DataFrame(sc.transform(train))
        test = pd.DataFrame(sc.transform(test))
        
        X_train, y_train = self.get_lag_response_and_dependent(train, input_size, horizon_size)
        X_test, y_test = self.get_lag_response_and_dependent(test, input_size, horizon_size)

        X_train = pd.DataFrame(X_train)
        y_train = pd.DataFrame(y_train)
        X_test = pd.DataFrame(X_test)
        y_test = pd.DataFrame(y_test)

        # Create hierarhical columns
        X_cols_lvl2 = [ f"t={i-input_size}" for i in range(0,input_size) for _ in range(len(self.columns))]
        X_cols_lvl1 = [col for _ in range(input_size) for col in self.columns]
        X_cols = pd.MultiIndex.from_tuples(zip(X_cols_lvl1, X_cols_lvl2))
        y_cols_lvl2 = [ f"t={i}" for i in range(0,horizon_size) for _ in range(len(self.columns))]
        y_cols_lvl1 = [col for _ in range(horizon_size) for col in self.columns]
        y_cols = pd.MultiIndex.from_tuples(zip(y_cols_lvl1, y_cols_lvl2))
        X_train.columns = X_cols
        y_train.columns = y_cols
        X_test.columns = X_cols
        y_test.columns = y_cols
        return X_train, y_train, X_test, y_test
        

# loads the ECL data in cut into smaller subframes
class ECLDataLoader:
    def __init__(self, filepath:str, j:int, col_number_per_split:int):
        self.filepath = filepath
        self.dataset_name = self.filepath.split("/")[-1].split(".")[0]
        self.j = j
        self.col_number_per_split = col_number_per_split
        self.df = self.load_data(self.filepath, self.j, self.col_number_per_split)
    
    def load_data(self, filepath:str, j:int, col_number_per_split:int) -> pd.DataFrame:
        df = pd.read_csv(filepath)
        df = df.drop(columns="date")

        num_cols = df.shape[1]
        # Determine the number of splits needed
        num_splits = num_cols // col_number_per_split   # Number of splits with 50 columns
        
        dfs = []
        for i in range(num_splits):
            dfs.append(df.iloc[:, i * col_number_per_split : (i + 1) * col_number_per_split])
        
        dfs.append(df.iloc[:, num_splits * col_number_per_split :])

        return dfs[j]
    

class ETTDataLoader(DataLoader):
    def __init__(self, filepath:str, months=20, day_length=24):
        self.filepath = filepath
        self.months = months
        self.dataset_name = self.filepath.split("/")[-1].split(".")[0]
        self.boundries = [0, day_length*16*30, day_length*20*30]
        self.df = self.load_data(self.filepath)
        self.columns = self.df.columns
        
    def load_data(self, filepath:str) -> pd.DataFrame:
        # Read file
        ett = pd.read_csv(filepath)
        ett = ett.drop(columns="date")
        
        #get correct rows
        ett = ett.iloc[0:self.boundries[2]]
        return ett
        
    def get_train_test_data(self, df:pd.DataFrame, input_size = 672, padding=True, day_length=20) -> Tuple[pd.DataFrame, pd.DataFrame]:
        tmp = df.copy()
        train = tmp.iloc[0: self.boundries[1]]
        test = tmp.iloc[self.boundries[1]:]
        if padding:
            test = pd.concat([train.iloc[-input_size:], test])
        return train, test

# Linear Regression Implementation

In [23]:
def run_LinearRegression_Experiment(dataLoaders, horizons, input_size=672):
    res = []
    lr = LinearRegression()
    dataset_names = []
    for j, loader in enumerate(dataLoaders):
        mse = np.zeros([1, len(horizons)])
        mae = np.zeros([1, len(horizons)])
        for i, h in enumerate(horizons):
            diffs = []
            for col in tqdm(loader.columns):
                X_train, y_train, X_test, y_test = loader.build_dataset(horizon_size=h, input_size=input_size)
                lr.fit(X_train[col], y_train[col])
                y_pred = lr.predict(X_test[col])
                diff = y_pred - y_test[col].values
                diffs += list(diff)
            diffs = np.array(diffs)
            mae[0,i] = np.mean(np.abs(diffs))
            mse[0,i] = np.mean(diffs**2)
        res.append(mae[0])
        res.append(mse[0])
    
    
    dataset_names = [l.dataset_name for l in dataLoaders for i in range(2)]
    metrics = ['mae', 'mse'] * len(dataLoaders)
    ml_index = pd.MultiIndex.from_arrays([dataset_names, metrics])
    return pd.DataFrame(res, index=ml_index, columns=horizons)

In [46]:
## ETTh
etth1DataLoader = ETTDataLoader("../exercise2/data/ETT-small-20231205T092053Z-001/ETT-small/ETTh1.csv")
etth2DataLoader = ETTDataLoader("../exercise2/data/ETT-small-20231205T092053Z-001/ETT-small/ETTh2.csv")

dataLoaders = [etth1DataLoader, etth2DataLoader]
horizons = [24, 48, 168, 336, 720]
regression_etth_res = run_LinearRegression_Experiment(dataLoaders, horizons)
regression_etth_res.to_csv("data/reproduced_results/etth_regression.csv")

## ETTm
ettm1DataLoader = ETTDataLoader("../exercise2/data/ETT-small-20231205T092053Z-001/ETT-small/ETTm1.csv")
dataLoaders = [ettm1DataLoader]
horizons = [24, 48, 96, 228, 672]
regression_ettm_res = run_LinearRegression_Experiment(dataLoaders, horizons)
regression_ettm_res.to_csv("data/reproduced_results/ettm_regression.csv")

## ILI
iliLoader = DataLoader("../exercise2/data/illness-20231205T092100Z-001/illness/national_illness.csv")
dataLoaders = [iliLoader]
horizons = [24, 36, 48, 60]
regression_ili_res = run_LinearRegression_Experiment(dataLoaders, horizons, input_size=96)
regression_ili_res.to_csv("data/reproduced_results/ili_regression.csv")

## WTH
wthLoader = DataLoader("../exercise2/data/WTH.csv-20231205T092445Z-001/WTH.csv")
dataLoaders = [wthLoader]
horizons = [24, 48, 168, 338, 720]
regression_wth_res = run_LinearRegression_Experiment(dataLoaders, horizons)
regression_wth_res.to_csv("data/reproduced_results/wth_regression.csv")

## Weather
weatherLoader = DataLoader("../exercise2/data/weather-20231205T093714Z-001/weather/weather.csv")
dataLoaders = [weatherLoader]
horizons = [96, 192, 336, 720]
regression_weather_res = run_LinearRegression_Experiment(dataLoaders, horizons)
regression_weather_res.to_csv("data/reproduced_results/weather_regression.csv")

# Exchange
exchange_rate = DataLoader("../exercise2/data/exchange_rate-20231205T092055Z-001/exchange_rate/exchange_rate.csv")
dataLoaders = [exchange_rate]
horizons = [96, 192, 336, 720]
regression_exchange_res = run_LinearRegression_Experiment(dataLoaders, horizons, input_size=31)
regression_exchange_res.to_csv("data/reproduced_results/exchange_regression.csv")

 10%|▉         | 2/21 [05:54<56:39, 178.94s/it]

In [ ]:
# ECL
ECL = DataLoader("../exercise2/data/ECL.csv-20231205T092501Z-001/ECL.csv")
dataLoaders = [ECL]
horizons = [48, 168, 336, 720, 960]
regression_ecl_res = run_LinearRegression_Experiment(dataLoaders, horizons)
regression_ecl_res.to_csv("data/reproduced_results/ecl_regression.csv")

# SNaive Implementation

In [34]:
def snaive_predict(series, season, horizon):
    # check if there are enough datapoints
    if len(series) < season:
        raise ValueError("Insufficient data for forecasting.")
    
    last_season_observation = series[-season:]

    prediction = np.tile(last_season_observation, horizon // season + 1)[:horizon]

    return prediction

def run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio):
    
    res = []
    dataset_names = []

    for j, loader in enumerate(dataLoaders):
        df = loader.df
        df = df.values

        num_rows, num_cols = df.shape

        num_train = round(num_rows * train_ratio) # round or cut or ...
        num_test = num_rows - num_train

        # normalisation -------------------------------------------------------------
        num_normalisation = round(num_train * normalisation_ratio)
        
        mean = df[:num_normalisation].mean(axis=0)
        std = df[:num_normalisation].std(axis=0)

        df_normalised = (df - mean) / std
        # ---------------------------------------------------------------------------
        mse = np.zeros([1, len(horizons)])
        mae = np.zeros([1, len(horizons)])

        for j, horizon in enumerate(horizons):
            diff_per_horizon = []

            for col in range(num_cols):
                diff_per_variable = []
                for i in range(num_test - horizon + 1):    # number of tests we can do in total
                    train_window = df_normalised[num_train + i - season:num_train + i,col]  
                    #print("last season values: ", train_window)
                    predict_window = snaive_predict(train_window, season, horizon)
                    #print("predicted values: ", predict_window)
                    test_window = df_normalised[num_train + i:num_train + i + horizon,col]  
                    #print("actual values: ", test_window)
                    
                    diff = np.array(test_window) - np.array(predict_window)  
                    diff_per_variable += list(diff) # diff_per_variable = [0.2, 0.5, 1.2, ..., 0.1] diff for one whole variable ~4000items for ili

                diff_per_horizon += list(diff_per_variable) # diff_per_variable = [0.2, 0.5, 1.2, ..., 0.7] diff for all variables ~28000items for ili
            mae[0,j] = np.mean(np.abs(diff_per_horizon))
            mse[0,j] = np.mean(np.square(diff_per_horizon))
        res.append(mae[0])
        res.append(mse[0])

    dataset_names = [l.dataset_name for l in dataLoaders for i in range(2)]
    metrics = ['mae', 'mse'] * len(dataLoaders)      #Joseph changed this line form this: [metric for metric in ("mae", "mse") for i in range(len(dataLoaders))]
    ml_index = pd.MultiIndex.from_arrays([dataset_names, metrics])
    
    return pd.DataFrame(res, index=ml_index, columns=horizons)

### Train ratio for ETT:

" For ETT, the data for 1-16 months is training data, and the data for 17-20 months is test data. "  

They calculated with 1month = 30days  
ETTDataLoader only selects the first 20 months  
Months 1-16 are train -> train_ratio = 0.8   

In [35]:
## ETTh
snaive_etth_res_list = [] # save the results of different seasons in the same list and create the df later

etth1DataLoader = ETTDataLoader("../exercise2/data/ETT-small-20231205T092053Z-001/ETT-small/ETTh1.csv")
etth2DataLoader = ETTDataLoader("../exercise2/data/ETT-small-20231205T092053Z-001/ETT-small/ETTh2.csv")
train_ratio = 0.8
normalisation_ratio = 0.75

dataLoaders = [etth1DataLoader, etth2DataLoader]
horizons = [24, 48]
season = 24
snaive_etth_res_list.append(run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio))

dataLoaders = [etth1DataLoader, etth2DataLoader]
horizons = [168, 336]
season = 168  # 24*7
snaive_etth_res_list.append(run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio))

dataLoaders = [etth1DataLoader, etth2DataLoader]
horizons = [720]
season = 672  # 24*28(28days=4weeks)
snaive_etth_res_list.append(run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio))

snaive_etth_res = pd.concat([snaive_etth_res_list[0], snaive_etth_res_list[1], snaive_etth_res_list[2]], axis=1)
snaive_etth_res.to_csv("data/reproduced_results/etth_snaive.csv")

## ETTm
ettm1DataLoader = ETTDataLoader("../exercise2/data/ETT-small-20231205T092053Z-001/ETT-small/ETTm1.csv")
dataLoaders = [ettm1DataLoader]
horizons = [24, 48, 96, 228, 672]
season = 96  # 24*4 (15min samples)
train_ratio = 0.8
normalisation_ratio = 0.75
snaive_ettm_res = run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio)
snaive_ettm_res.to_csv("data/reproduced_results/ettm_snaive.csv")

## ILI
iliLoader = DataLoader("../exercise2/data/illness-20231205T092100Z-001/illness/national_illness.csv")
dataLoaders = [iliLoader]
horizons = [24, 36, 48, 60]
season = 52
train_ratio = 0.8
normalisation_ratio = 0.875
snaive_ili_res = run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio)
snaive_ili_res.to_csv("data/reproduced_results/ili_snaive.csv")

## WTH
wthLoader = DataLoader("../exercise2/data/WTH.csv-20231205T092445Z-001/WTH.csv")
dataLoaders = [wthLoader]
horizons = [24, 48, 168, 338, 720]
season = 24
train_ratio = 0.8
normalisation_ratio = 0.875
snaive_wth_res = run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio)
snaive_wth_res.to_csv("data/reproduced_results/wth_snaive.csv")

## Weather
weatherLoader = DataLoader("../exercise2/data/weather-20231205T093714Z-001/weather/weather.csv")
dataLoaders = [weatherLoader]
horizons = [96, 192, 336, 720]
season = 144    # 24*6 (sample all 10min with daily season)
train_ratio = 0.8
normalisation_ratio = 0.875
snaive_weather_res = run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio)
snaive_weather_res.to_csv("data/reproduced_results/weather_snaive.csv")

# Exchange
exchange_rate = DataLoader("../exercise2/data/exchange_rate-20231205T092055Z-001/exchange_rate/exchange_rate.csv")
dataLoaders = [exchange_rate]
horizons = [96, 192, 336, 720]
season = 1
train_ratio = 0.8
normalisation_ratio = 0.875
snaive_exchange_res = run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio)
snaive_exchange_res.to_csv("data/reproduced_results/exchange_snaive.csv")

### Runtime error with ECL

We can divide the dataframe into equaly large subsets, calculate their mean and then take the mean of the means of the same.
The dataframe has 322 - date = 321 columns. When doing that we should not have a problem when calculating.  
The divisors of 321 are 1, 3, 107, 321. Dividing by 3 would be a bit small and dividing by 107 a bit large. Therefore we make subsets of 50columns, multiply the mean of their columns by col_number_per_split/321 and add them all up:

### Solution: ECL split into multiple files

In [36]:
#ECL
horizons = [48, 168, 336, 720, 960]
season = 168    # 24*7 (sample hourly with weekly season)
train_ratio = 0.8
normalisation_ratio = 0.875
snaive_ecl_res_list = []
col_number_per_split = 50
total_number_of_columns = 321

In [37]:
ECL = ECLDataLoader("../exercise2/data/ECL.csv-20231205T092501Z-001/ECL.csv", 0, col_number_per_split) #columns 0-49
dataLoaders = [ECL]
snaive_ecl_res_list.append(run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio).multiply(col_number_per_split/total_number_of_columns))

In [38]:
ECL = ECLDataLoader("../exercise2/data/ECL.csv-20231205T092501Z-001/ECL.csv", 1, col_number_per_split) #columns 50-99
dataLoaders = [ECL]
snaive_ecl_res_list.append(run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio).multiply(col_number_per_split/total_number_of_columns))

In [39]:
ECL = ECLDataLoader("../exercise2/data/ECL.csv-20231205T092501Z-001/ECL.csv", 2, col_number_per_split) #columns 100-149
dataLoaders = [ECL]
snaive_ecl_res_list.append(run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio).multiply(col_number_per_split/total_number_of_columns))

In [40]:
ECL = ECLDataLoader("../exercise2/data/ECL.csv-20231205T092501Z-001/ECL.csv", 3, col_number_per_split) #columns 150-199
dataLoaders = [ECL]
snaive_ecl_res_list.append(run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio).multiply(col_number_per_split/total_number_of_columns))

In [41]:
ECL = ECLDataLoader("../exercise2/data/ECL.csv-20231205T092501Z-001/ECL.csv", 4, col_number_per_split) #columns 200-249
dataLoaders = [ECL]
snaive_ecl_res_list.append(run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio).multiply(col_number_per_split/total_number_of_columns))

In [42]:
ECL = ECLDataLoader("../exercise2/data/ECL.csv-20231205T092501Z-001/ECL.csv", 5, col_number_per_split) #columns 250-299
dataLoaders = [ECL]
snaive_ecl_res_list.append(run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio).multiply(col_number_per_split/total_number_of_columns))

In [43]:
ECL = ECLDataLoader("../exercise2/data/ECL.csv-20231205T092501Z-001/ECL.csv", 6, col_number_per_split) #columns 300-320
dataLoaders = [ECL]
snaive_ecl_res_list.append(run_SNaive_Experiment(dataLoaders, horizons, season, train_ratio, normalisation_ratio).multiply((total_number_of_columns%col_number_per_split)/total_number_of_columns))

In [44]:
snaive_ecl_res = None

for dataframe in snaive_ecl_res_list:
    if snaive_ecl_res is None:
        snaive_ecl_res = dataframe.copy()
    else:
        snaive_ecl_res = snaive_ecl_res.add(dataframe)

snaive_ecl_res.to_csv("data/reproduced_results/ecl_snaive.csv")